# Modello convolutivo di base
In questo notebook costruiamo un **modello convolutivo di base** così da poter avere un'idea migliore del task che stiamo approcciando. Utilizzando un modello di base riusciamo a capire soprattutto che prestazioni riusciamo a ottenere senza troppi sforzi.

In [ ]:
!pip install tensorflow_io

In [34]:
import tensorflow as tf
import tensorflow_io as tfio
import matplotlib.pyplot as plt

In [93]:
train_ds, validation_ds = tf.keras.utils.audio_dataset_from_directory(
    directory='../reduced_dataset/dataset/audio', 
    validation_split=0.4, # stiamo mettendo da parte il 40% del dataset, che sarà suddiviso in validation set e test set
    shuffle=True, 
    subset='both', # necessario se stiamo utilizzando validation_split (se no darebbe errore)
    seed=0 # necessario se stiamo utilizzando sia shuffle che validation_split (se no darebbe errore)
)

Found 12933 files belonging to 30 classes.
Using 7760 files for training.
Using 5173 files for validation.


In [94]:
for audios, labels in train_ds:
    print("Forma (shape) audio:", audios.shape)
    print("Forma (shape) classi:", labels.shape)
    break

Forma (shape) audio: (32, 16000, 1)
Forma (shape) classi: (32,)


Possiamo notare che lo shape dell'audio corrisponde a (32, 16000, 1), questo vuol dire che ogni "audio" che stiamo stampando in realtà è un batch da 32 tensori di lunghezza 16000 unidimensionali. Ovviamente anche lo shape delle classi sarà 32 visto e considerata la presenza di 32 tensori per iterazione. Abbiamo perciò capito che il nostro dataset è suddiviso in batch da 32 di grandezza, questo perché di defau

In [87]:
def squeeze(audio, labels): 
    audio = tf.squeeze(audio, axis=-1) 
    return audio, labels 
  
train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE) 
validation_ds = validation_ds.map(squeeze, tf.data.AUTOTUNE)

# fonte: funzione presa dal seguente link: https://www.geeksforgeeks.org/audio-recognition-in-tensorflow/

In [ ]:
def get_spectrogram(waveform): 
    spectrogram = tf.signal.stft(waveform, frame_length=255, frame_step=128) 
    spectrogram = tf.abs(spectrogram) 
    
    return spectrogram[..., tf.newaxis] 

# fonte: funzione presa dal seguente link: https://www.geeksforgeeks.org/audio-recognition-in-tensorflow/

In [86]:
def get_spectrogram_dataset(dataset):
    dataset = dataset.map(lambda x, y: (get_spectrogram(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    return dataset

# fonte: funzione presa dal seguente link: https://www.geeksforgeeks.org/audio-recognition-in-tensorflow/